In [ ]:
!pip install telethon

In [ ]:
pip install -q -U google-genai

In [ ]:
import nest_asyncio
import asyncio
import logging
from telethon import TelegramClient, events, Button
from google import genai

# Enable nest_asyncio to avoid event loop conflicts in Jupyter or Colab
nest_asyncio.apply()

# Define necessary credentials
api_id = "21403141"
api_hash = "74e9c96971cd422c1ba9878a13ea3db1"
bot_token = "7147482273:AAH0FlgwrCSuRuyikXzELEljztMn_Sb-zeo"

# Initialize the client
client = TelegramClient('bot', api_id, api_hash)

# Initialize Google GenAI Client
genai_client = genai.Client(api_key="YOUR_API_KEY")

# Define main function
async def main():
    await client.start(bot_token=bot_token)

    @client.on(events.NewMessage(pattern='/start'))
    async def start_handler(event):
        user = await client.get_entity(event.sender_id)
        first_name = user.first_name if user.first_name else "there"

        await event.respond(
            f"\U0001F44B Hello {first_name}! I am EdgeAI from HyperVision. How can I assist you today? \U0001F916",
            buttons=[
                [Button.url("\U0001F310 Visit our website", "https://example.com")],
                [Button.url("ℹ️ Learn more about me", "https://example.com/info")],
                [Button.url("📢 Join our community", "https://t.me/your_channel")]
            ]
        )
        logging.info(f'Start command received from {event.sender_id}')

    @client.on(events.NewMessage(pattern='/info'))
    async def info_handler(event):
        await event.respond("🤖 I am EdgeAI from HyperVision, developed by Wail Achouri. 🎯")
        logging.info(f'Info command received from {event.sender_id}')

    @client.on(events.NewMessage(pattern='/help'))
    async def help_handler(event):
        help_text = (
            "🛠️ **Available Commands:**\n"
            "/start - Start the bot 🚀\n"
            "/help - Get help 📖\n"
            "/info - Learn more about me ℹ️\n"
            "/generate_image description - Generate an AI image 🎨"
        )
        await event.respond(help_text)
        logging.info(f"Help command received from {event.sender_id}")

    @client.on(events.NewMessage(pattern='/generate_image (.+)'))
    async def image_handler(event):
        user = await client.get_entity(event.sender_id)
        first_name = user.first_name if user.first_name else "Friend"
        prompt = event.pattern_match.group(1)

        await event.respond(f"🎨 Generating your image, {first_name}... Please wait. ⏳")

        try:
            async with client.action(event.chat_id, 'photo'):
                image_generation = await asyncio.to_thread(
                    genai_client.models.generate_content,
                    model="gemini-2.0-flash",
                    contents=prompt
                )

                # Check if the response is valid
                if image_generation and hasattr(image_generation, 'text'):
                    image_url = image_generation.text  # Assuming the API returns a URL or path to the generated image
                    await client.send_file(event.chat_id, image_url, caption=f"✨ Here is your image, {first_name}! 🎨")
                else:
                    await event.respond("⚠️ Sorry, I couldn't generate the image. The response was empty.")

        except Exception as e:
            logging.error(f"Error generating image: {e}")
            await event.respond("⚠️ Sorry, I couldn't generate the image. Please try again later.")

    @client.on(events.NewMessage)
    async def keyword_responder(event):
        message = event.text.lower()
        if message.startswith('/'):
            return

        user = await client.get_entity(event.sender_id)
        first_name = user.first_name if user.first_name else "Friend"

        try:
            async with client.action(event.chat_id, 'typing'):
                instructions = """
                You are EdgeAI, an advanced AI developed by HyperVision and created by Wail Achouri.
                Your responses should be short, fast, and include relevant emojis.
                Always personalize responses using the user's first name if available.
                """

                # Request content generation from Google GenAI
                completion = await asyncio.to_thread(
                    genai_client.models.generate_content,
                    model="gemini-2.0-flash",
                    contents=f"The user's name is {first_name} and they asked: {message}"
                )

                if completion and hasattr(completion, 'text'):
                    response = completion.text
                    await event.respond(response)
                else:
                    await event.respond("⚠️ Sorry, I couldn't process your request right now.")

        except Exception as e:
            logging.error(f"Error while fetching response: {e}")
            await event.respond("⚠️ Sorry, I couldn't process your request right now.")

    await client.run_until_disconnected()

# Run the main function
await main()